In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
torch.manual_seed(42)


In [2]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('../Data/research-abstracts-labeled.csv')
gpt = df[df['label'] == 1]
human = df[df['label'] == 0]


In [4]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [5]:
gpt['embeddings'] = gpt['text'].apply(generate_bert_embeddings)
gpt['normalized_embeddings'] = gpt['embeddings'].apply(remove_outliers)
gpt['normalized_embeddings1'] = gpt['normalized_embeddings'].apply(min_max_normalize)
gpt['reshaped_embeddings'] = gpt['normalized_embeddings1'].apply(reshape_embeddings)

/tmp/ipykernel_915780/2595106749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt['embeddings'] = gpt['text'].apply(generate_bert_embeddings)
/tmp/ipykernel_915780/2595106749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt['normalized_embeddings'] = gpt['embeddings'].apply(remove_outliers)
/tmp/ipykernel_915780/2595106749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [11]:
gpt_embeddings = gpt['embeddings'].to_numpy()

In [12]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/gpt_embeddings_research.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, gpt_embeddings)
    

In [8]:
human['embeddings'] = human['text'].apply(generate_bert_embeddings)
human['normalized_embeddings'] = human['embeddings'].apply(remove_outliers)
human['normalized_embeddings1'] = human['normalized_embeddings'].apply(min_max_normalize)
human['reshaped_embeddings'] = human['normalized_embeddings1'].apply(reshape_embeddings)

/tmp/ipykernel_915780/3091784679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human['embeddings'] = human['text'].apply(generate_bert_embeddings)
/tmp/ipykernel_915780/3091784679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human['normalized_embeddings'] = human['embeddings'].apply(remove_outliers)
/tmp/ipykernel_915780/3091784679.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [9]:
human_embeddings = human['embeddings'].to_numpy()

In [13]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/human_embeddings_research.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, human_embeddings)
    